In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input, LSTM
from gensim.models import Word2Vec
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
%matplotlib inline


# nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
df = pd.read_csv('/content/drive/MyDrive/NLP/Corona_NLP_train.csv', encoding = "ISO-8859-1")
df

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative
...,...,...,...,...,...,...
41152,44951,89903,"Wellington City, New Zealand",14-04-2020,Airline pilots offering to stock supermarket s...,Neutral
41153,44952,89904,NaN,14-04-2020,Response to complaint not provided citing COVI...,Extremely Negative
41154,44953,89905,NaN,14-04-2020,You know itÂs getting tough when @KameronWild...,Positive
41155,44954,89906,NaN,14-04-2020,Is it wrong that the smell of hand sanitizer i...,Neutral


In [38]:
sample_str = df.OriginalTweet.values[0]

print('== Исходный текст== \n%s\n\n' % sample_str)

tokenized_str = word_tokenize(sample_str)
print('== Токенизированный текст==\n%s' % tokenized_str)

== Исходный текст== 
@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/iFz9FAn2Pa and https://t.co/xX6ghGFzCC and https://t.co/I2NlzdxNo8


== Токенизированный текст==
['@', 'MeNyrbie', '@', 'Phil_Gahan', '@', 'Chrisitv', 'https', ':', '//t.co/iFz9FAn2Pa', 'and', 'https', ':', '//t.co/xX6ghGFzCC', 'and', 'https', ':', '//t.co/I2NlzdxNo8']


In [39]:
tokenized_str = np.array( tokenized_str )
idx_r = np.where(tokenized_str == '@')[0]
idx_s = np.where( tokenized_str == 'https' )[0]

In [25]:
idx_r = np.append(idx_r, [idx_r + 1])
idx_s = np.append(idx_s, [idx_s + 1, idx_s + 2])
idxs = np.concatenate([idx_r, idx_s])
idxs

array([5, 6, 7])

In [41]:
tokenized_str = np.delete(tokenized_str, [idxs])
tokenized_str

array(['Phil_Gahan', 'https', '//t.co/iFz9FAn2Pa', 'and', 'https', ':',
       '//t.co/xX6ghGFzCC', 'and', 'https', ':', '//t.co/I2NlzdxNo8'],
      dtype='<U17')

In [27]:
tokens = [i.lower() for i in tokenized_str if ( i not in punctuation )]
print(tokens)

['come', 'on', 'chickens']


In [22]:
stop_words = set(stopwords.words('english'))
filtered_tokens = [i for i in tokens if ( i not in stop_words )]
filtered_tokens

['shame']

In [36]:
df.tokin[0]

'@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/iFz9FAn2Pa and https://t.co/xX6ghGFzCC and https://t.co/I2NlzdxNo8'

In [98]:
stop_words = set(stopwords.words('english'))
def tokenize_text(raw_text: str):
    """Функция для токенизации текста
    
    :param raw_text: исходная текстовая строка
    """
    tokenized_str = np.array( word_tokenize(raw_text) )
    idx_r = np.where( tokenized_str == '@' )[0]
    idx_s = np.where( tokenized_str == 'https' )[0]

    idx_r = np.append(idx_r, [idx_r + 1])
    idx_s = np.append(idx_s, [idx_s + 1, idx_s + 2])
    idxs = np.concatenate([idx_r, idx_s])
    
    try:
        tokenized_str = np.delete(tokenized_str, idxs)
    except:
        pass

    tokens = [i.lower() for i in tokenized_str if ( i not in punctuation )]
    # filtered_tokens = [i for i in tokens if ( i not in stop_words )]
    if tokens == []:
        return None
    return tokens

# применяем функцию в датафрейму с помощью метода .apply()
tokenized_tweets = df.OriginalTweet.apply(tokenize_text)

# добавляем новую колонку в исходный датафрейм
df = df.assign(
    tokenized=tokenized_tweets
)
df.dropna(subset = ['tokenized'], inplace = True)
df.reset_index()

df.tokenized.head()

0                                           [and, and]
1    [advice, talk, to, your, neighbours, family, t...
2    [coronavirus, australia, woolworths, to, give,...
5    [as, news, of, the, regionâs, first, confirme...
6    [cashier, at, grocery, store, was, sharing, hi...
Name: tokenized, dtype: object

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

vectorize = TfidfVectorizer().fit(df.tokenized)
vectorize.vocabulary_